In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime as DT
import pickle
print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [2]:
# read in Indicator data into dataframe
parser = lambda date: pd.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
df_Ind = pd.read_csv('../data/150001_Indicators.csv', parse_dates=[0], date_parser=parser)
#rename 'Date' column to 'DateTime' and split apart
df_Ind.rename(columns = {'Date': 'DateTime'}, inplace=True)
temp = pd.DatetimeIndex(df_Ind['DateTime'])
df_Ind['Date'] = temp.date
df_Ind['Time'] = temp.time

# rearange to put 'Date' and 'Time' up front behind 'DateTime'
cols = df_Ind.columns.tolist()
cols = cols[0:1] + cols[-2:] + cols[1:-2]
#cols
df_Ind = df_Ind[cols]
df_Ind.head()

,DateTime,Date,Time,Ticket,Open/Close,BBAND(10),BBAND(20),BBAND(30),BBAND(40),BBAND(50),...,MACD(12) MAIN,MACD(12) SIGNAL,MACD(14) MAIN,MACD(14) SIGNAL,MACD(16) MAIN,MACD(16) SIGNAL,MACD(18) MAIN,MACD(18) SIGNAL,MACD(20) MAIN,MACD(20) SIGNAL
0,2015-01-02 08:00:00,2015-01-02,08:00:00,-1,ON_NewBar,1.209722,1.210053,1.210167,1.210231,1.210213,...,-0.000425,-0.000060,-0.000425,-0.000047,-0.000425,-0.000036,-0.000425,-0.000026,-0.000425,-0.000016
1,2015-01-02 08:01:00,2015-01-02,08:01:00,-1,ON_NewBar,1.209193,1.209771,1.209984,1.210093,1.210109,...,-0.000757,-0.000126,-0.000757,-0.000105,-0.000757,-0.000088,-0.000757,-0.000073,-0.000757,-0.000059
2,2015-01-02 08:02:00,2015-01-02,08:02:00,-1,ON_NewBar,1.208657,1.209489,1.209802,1.209957,1.210005,...,-0.001009,-0.000211,-0.001009,-0.000180,-0.001009,-0.000154,-0.001009,-0.000132,-0.001009,-0.000113
3,2015-01-02 08:03:00,2015-01-02,08:03:00,-1,ON_NewBar,1.208082,1.209193,1.209604,1.209807,1.209895,...,-0.001229,-0.000315,-0.001229,-0.000271,-0.001229,-0.000234,-0.001229,-0.000204,-0.001229,-0.000179
4,2015-01-02 08:04:00,2015-01-02,08:04:00,-1,ON_NewBar,1.207358,1.208821,1.209358,1.209620,1.209755,...,-0.001498,-0.000444,-0.001498,-0.000383,-0.001498,-0.000334,-0.001498,-0.000294,-0.001498,-0.000260


In [3]:
# read in order data into dataframe
df_ord = pd.read_csv('../data/150001_Orders.csv', parse_dates=[1], date_parser=parser)

#rename 'Date' column to 'DateTime' and split apart
df_ord.rename(columns = {'Open Time': 'DateTime'}, inplace=True)
temp = pd.DatetimeIndex(df_ord['DateTime'])
df_ord['Date'] = temp.date
df_ord['Time'] = temp.time

# rearange to put 'Date' and 'Time' up front behind 'DateTime'
cols = df_ord.columns.tolist()
cols = cols[1:2] + cols[-2:] + cols[0:1] + cols[2:-2]
#cols
df_ord = df_ord[cols]
df_ord.head()

,DateTime,Date,Time,Ticket,Type,Lots,Symbol,Open Price,StopLoss,TakeProfit,Close Time,Close Price,Commission,Swap,Profit,Comment,MagicNumber,Expiration
0,2015-01-04 23:41:30,2015-01-04,23:41:30,1,OP_SELL,0.1,EURUSD,1.19540,1.19940,1.19040,2015.01.05 00:15:40,1.19360,0,-0.15,18.0,Sell,150001,1970.01.01 00:00:00
1,2015-01-06 09:51:40,2015-01-06,09:51:40,2,OP_SELL,0.1,EURUSD,1.19392,1.19792,1.18892,2015.01.06 10:20:40,1.19293,0,0.00,9.9,Sell,150001,1970.01.01 00:00:00
2,2015-01-06 12:04:59,2015-01-06,12:04:59,3,OP_SELL,0.1,EURUSD,1.19122,1.19522,1.18622,2015.01.06 12:16:59,1.18954,0,0.00,16.8,Sell,150001,1970.01.01 00:00:00
3,2015-01-07 13:20:40,2015-01-07,13:20:40,4,OP_SELL,0.1,EURUSD,1.18432,1.18832,1.17932,2015.01.07 13:52:40,1.18318,0,0.00,11.4,Sell,150001,1970.01.01 00:00:00
4,2015-01-08 04:07:00,2015-01-08,04:07:00,5,OP_BUY,0.1,EURUSD,1.18198,1.17798,1.18698,2015.01.08 11:02:18,1.17798,0,0.00,-40.0,Buy[sl],150001,1970.01.01 00:00:00


In [4]:
# creation of graph to viaully determine trend ranges
# plot rolling data, intent was to determine range of UP, DOWN and FLAT for nex step
#plt.plot(df_ord['Ticket'], df_ord['Roll_std'])
#plt.plot(df_ord['Ticket'], df_ord['Profit'], marker='x', linestyle=':')

#plt.show()

# Create Trend Ranges, based on visual inspection of previous graph, and add Trends to Order dataframe
trend_range = [0, 6, 33, 60, 77, 86, 150, 171, 207, 222, 271, 314, 331, 348]
trend_labels = ['UP1', 'FLAT1', 'DOWN1', 'UP2', 'DOWN2', 'FLAT2', 'DOWN3', 'UP3', 'DOWN4', 'UP4', 
               'DOWN5', 'UP6', 'FLAT3']

#Categorize order dataframe based on trend ranges
df_ord['Trend'] = pd.cut(df_ord.Ticket, trend_range, labels=trend_labels).astype('category')

In [5]:
#
# FUCNTION DEFINITIONS
#

# function returns sum of count of number of profit (+1) and loss trades (-1)
def rolling_profit_count(dataframe):
    count = 0
    for profit in dataframe:
        if (profit >= 0):
            count = count + 1
        else:
            count = count -1
    return count


# function to convert 'DateTime' to seconds
def seconds(datetime):
    return (datetime - DT.datetime(1970, 1, 1)).total_seconds()

In [6]:
# Add Rolling stats to Order DF
window = 15
df_ord['Roll_Profit_Count'] = pd.rolling_apply(df_ord['Profit'], window, rolling_profit_count,1)
df_ord['Roll_Mean'] = pd.rolling_mean(df_ord['Profit'], window)
df_ord['Roll_std'] = pd.rolling_std(df_ord['Profit'], window)
df_ord['Roll_var'] = pd.rolling_var(df_ord['Profit'], window)
# Order Dataframe with Trends added
df_ord[['DateTime', 'Ticket', 'Profit', 'Trend', 'Roll_Profit_Count', 'Roll_Mean', 'Roll_std']].head()

,DateTime,Ticket,Profit,Trend,Roll_Profit_Count,Roll_Mean,Roll_std
0,2015-01-04 23:41:30,1,18.0,UP1,1,NaN,NaN
1,2015-01-06 09:51:40,2,9.9,UP1,2,NaN,NaN
2,2015-01-06 12:04:59,3,16.8,UP1,3,NaN,NaN
3,2015-01-07 13:20:40,4,11.4,UP1,4,NaN,NaN
4,2015-01-08 04:07:00,5,-40.0,UP1,3,NaN,NaN


In [7]:
# Convert Date time in Order DF to seconds
df_ord['DateTime_sec'] = df_ord['DateTime'].apply(seconds)

In [8]:
# Convert Date time in Order DF to seconds
df_ord['DateTime_sec'] = df_ord['DateTime'].apply(seconds)

# pull 'Times' of trend range changes from order dataframe
# i.e. if 'Ticket' is in trend_range list, return the 'DateTime'
trend_range_DT = df_ord[df_ord['Ticket'].isin(trend_range)]['DateTime']
trend_range_DT_sec = df_ord[df_ord['Ticket'].isin(trend_range)]['DateTime_sec']

#Add the 'first ticket time' to the Series
zerotime = int(df_ord['DateTime_sec'][0])
trend_range_DT_sec = trend_range_DT_sec.append(pd.Series([zerotime])).sort_values(ascending=True)
zerotime = df_ord['DateTime'][0]
trend_range_DT = trend_range_DT.append(pd.Series([zerotime])).sort_values(ascending=True)

# Convert Date time in Indicator DF to seconds and pull that Series out
df_Ind['DateTime_sec'] = df_Ind['DateTime'].apply(seconds)

#Categorize Indicator dataframe based on trend ranges
df_Ind['Trend'] = pd.cut(df_Ind.DateTime_sec, trend_range_DT_sec, labels=trend_labels).astype('category')

In [9]:
# Create dataframe study to contain data that will be evaluated
study = df_Ind.loc[:, ('DateTime', 'Trend', 'BBAND(100)', 'MACD(12) MAIN', 'MACD(2) SIGNAL', 'MACD(10) SIGNAL', 'MACD(12) SIGNAL' , 'MACD(20) SIGNAL')]

# Add stats to study dataframe
rolling_window = 7
study['bband100_std'] = pd.rolling_std(study['BBAND(100)'], rolling_window)
study['macd12main_std'] = pd.rolling_std(study['MACD(12) MAIN'], rolling_window)
study['macd2signal_std'] = pd.rolling_std(study['MACD(2) SIGNAL'], rolling_window)
study['mcad2signal_std_ave'] = pd.rolling_mean(study['macd2signal_std'], rolling_window)
study['macd10signal_std'] = pd.rolling_std(study['MACD(10) SIGNAL'], rolling_window)
study['macd12signal_std'] = pd.rolling_std(study['MACD(12) SIGNAL'], rolling_window)
study['mcad12signal_std_ave'] = pd.rolling_mean(study['macd12signal_std'], rolling_window*3)
study['macd20signal_std'] = pd.rolling_std(study['MACD(20) SIGNAL'], rolling_window)

In [16]:
#define Gain and Loss filters
Gain = (df_ord['Profit'] > 0)
Loss = (df_ord['Profit'] < 0)

#define filters for trends
up1 = (study['Trend'] == 'UP1')
flat1 = (study['Trend'] == 'FLAT1')
down1 = (study['Trend'] == 'DOWN1')
up2 = (study['Trend'] == 'UP2')
down2 = (study['Trend'] == 'DOWN2')
flat2 = (study['Trend'] == 'FLAT2')
down3 = (study['Trend'] == 'DOWN3')
up3 = (study['Trend'] == 'UP3')
down4 = (study['Trend'] == 'DOWN4')
up4 = (study['Trend'] == 'UP4')
down5 = (study['Trend'] == 'DOWN5')
up6 = (study['Trend'] == 'UP6')
flat3 = (study['Trend'] == 'FLAT3')

In [17]:
# Pull Gain orders from Order dataframe
gain = df_ord.loc[Gain, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]
gain['PlotLevel'] = 0.00020
loss = df_ord.loc[Loss, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]
loss['PlotLevel'] = 0.00020

In [61]:
def plotter(plotdata, **kwarg):

    # iterate through argument list and assign appropriate chart data
    for kw, val in kwarg.items():
        color = 'blue'
        wt = 1
        zorder = 1
        linestyle = '-'
        marker = None
        if kw == 'sub':
            plt.subplot(val)
        if kw == 'ylabel':
            plt.ylabel(val)
        if kw == 'xlabel':
            plt.xlabel(val)
        if kw == 'ymax':
            plt.ylim(ymax=val)
        if kw == 'title':
            plt.title(val) 
    #iterate through plot data and get information for each series to be plotted
    for series, itemlist in plotdata.items():
        color = 'blue'
        wt = 1
        zorder = 1
        linestyle = '-'
        marker = None
        for item, val in itemlist.items():
            if item == 'xdata':
                xdata = val
            if item == 'ydata':
                ydata = val
            if item == 'dataframe':
                data = val   
            if item == 'color':
                color = val
            if item == 'wt':
                wt = val
            if item == 'zorder':
                zorder = val
            if item == 'linestyle':
                linestyle = val
            if item == 'marker':
                marker = val
        xmin, xmax = plt.xlim()
        #xmin = float(int(xmin))
        #xmax = float(int(xmax+1))
        print (data[xdata][0])
        #plt.xlim(xmin, xmax)
        plt.plot(list(data[xdata]),list(data[ydata]), color=color, zorder=zorder, marker=marker, linestyle=linestyle)
        

In [39]:
def plot(datarange, series, trend, subplot, color):
    plotter({'Series' + series:{
             'xdata':'DateTime',
             'ydata':series,
             'dataframe':datarange,
             'color':color},
         
             'Series:Loss':{
             'xdata':'DateTime',  
             'ydata':'PlotLevel',
             'dataframe':loss.loc[(loss['Trend'] == trend)],
             'color':'r', 'marker':'o', 'linestyle':''}, 
         
             'Series:Gain':{
             'xdata':'DateTime',
             'ydata':'PlotLevel',
             'dataframe':gain.loc[(gain['Trend'] == trend)],
             'color':'g', 'marker':'o', 'linestyle':''}}, 

             ylabel='Period '+ trend, sub=subplot, ymax=maxy, title='MACD (12) Main - STD')    

In [62]:
maxy = 0.0004
nrows = 4
ncols = 2
sub = (nrows * 100) + (ncols * 10)

plt.figure(1)
# plotter sub 1
plot(study[(study['Trend'] == 'UP1')], 'macd12main_std', 'UP1', sub+1, 'b')
# plotter sub 2
plot(study[(study['Trend'] == 'UP1')], 'macd12signal_std', 'UP1', sub+2, 'r')
# plotter sub 3
plot(study[(study['Trend'] == 'UP2')], 'macd12main_std', 'UP2', sub+3, 'b')
# plotter sub 4
plot(study[(study['Trend'] == 'UP2')], 'macd12signal_std', 'UP2', sub+4, 'r')
# plotter sub 5
plot(study[(study['Trend'] == 'UP3')], 'macd12main_std', 'UP3', sub+5, 'b')
# plotter sub 6
plot(study[(study['Trend'] == 'UP3')], 'macd12signal_std', 'UP3', sub+6, 'r')
# plotter sub 7
plot(study[(study['Trend'] == 'UP4')], 'macd12main_std', 'UP4', sub+7, 'b')
# plotter sub 8
plot(study[(study['Trend'] == 'UP4')], 'macd12signal_std', 'UP4', sub+8, 'r')

plt.figure(2)
# plotter sub 1
plot(study[(study['Trend'] == 'DOWN1')], 'macd12main_std', 'DOWN1', sub+1, 'b')
# plotter sub 2
plot(study[(study['Trend'] == 'DOWN1')], 'macd12signal_std', 'DOWN1', sub+2, 'r')
# plotter sub 3
plot(study[(study['Trend'] == 'DOWN2')], 'macd12main_std', 'DOWN2', sub+3, 'b')
# plotter sub 4
plot(study[(study['Trend'] == 'DOWN2')], 'macd12signal_std', 'DOWN2', sub+4, 'r')
# plotter sub 5
plot(study[(study['Trend'] == 'DOWN3')], 'macd12main_std', 'DOWN3', sub+5, 'b')
# plotter sub 6
plot(study[(study['Trend'] == 'DOWN3')], 'macd12signal_std', 'DOWN3', sub+6, 'r')
# plotter sub 7
plot(study[(study['Trend'] == 'DOWN4')], 'macd12main_std', 'DOWN4', sub+7, 'b')
# plotter sub 8
plot(study[(study['Trend'] == 'DOWN4')], 'macd12signal_std', 'DOWN4', sub+8, 'r')

plt.show()

KeyError: 0

In [36]:
plt.figure?